# HDS-LEE Course on Hyperparameter Optimization - Part 2

<img src='https://raw.githubusercontent.com/DLR-SC/Hyperparameter_tutorial/master/img/hds_lee_title.png' width=500px>

This notebook summarizes several examples from the Talos GitHub repository. The original examples can be found
on https://github.com/autonomio/talos/tree/master/examples

In [ ]:
%matplotlib inline
# required Talos __version__ = "0.6.3"
import talos
import numpy as np

## Overview

The aim of the 2nd tutorial is the following:

* Introduction of Talos
* Use Talos for machine-assisted hyperparameter optimization of the Boston house pricing problem (1st Jupyter notebook)
* Give general guidelines on hyperparameter optimization

### Table of Contents

##### 1. <a href=#one>Introduction of Talos</a>
##### 2. <a href=#two>Use Talos for hyperparameter optimization </a>
##### 3. <a href=#three>Guidelines on hyperparameter optimization</a>

## 1. Introduction of Talos <a name="one"></a> 

### Restore the original Keras code 

We start with the code that we have already used in the first notebook

In [ ]:
from keras.datasets import boston_housing

# load the data
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

# data normalization
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

### Model preparation

Talos works with any Keras model, without changing the structure of the model in any way.  
The only difference in the Keras model is that a parameter is not set explicitly as before, e.g.

<pre><code> model.add(layers.Dense(32, activation='relu'))    </code></pre>

but instead taken from a dictionary, e.g.

<pre><code>
params = {'number_of_neurons' : [4, 6, 7, 8], 'activation' : ['relu'] }

model.add(layers.Dense(params['number_of_neurons'], activation=params['activation']))    
</code></pre>

Afterwards, this dictionary and the model will be passed to Talos. In the dictionary we have 
three different ways to input values:

- as stepped ranges (min, max, steps)
- as multiple values [in a list]
- as a single value [in a list]

For values we don't want to use, it's ok to set it as None.

## Tasks ## 
__Excercise 1:__
 - Create a Python dictionary that contains at least the following entries:
    * 'number_of_layers' : 1, 2
    * 'number_of_neurons' : 8, 16, 32, 64
    * 'dropout_value' : None, 0.1, 0.2
    * 'optimizer' : 'Adam', 'rmsprop'
    * 'batch_size': 1, 2, 4, 8 
    * 'epoch_number' : 10, 20, 40, 80
    * anything that you want to modify further (e.g. learning_rate, activation_function, loss_function, ...)
 - What is the number of different hyperparameter configurations that is considered in the first six bullet points?

In [ ]:
# parameter dictionary
param = {
        # enter your code here
}


## Tasks ## 
__Excercise 2:__
 - Modify your original Keras model (Jupyter notebook 1) such that it uses uses the values from a 
 dictionary 'p'.

In [ ]:
from keras import models
from keras import layers

# note: this build model works for a limited number of hidden layers, there are other options in Talos
# if you are interested in a large number of layers (see: Outlook at the end of this tutorial)
def build_better_model(x, y, val_data, val_targets, p):
    model = models.Sequential()
    
    # replace the hyperparameter inputs with references to dictionary p 
    
    # enter your code here
        
    model.add(layers.Dense(1))
    model.compile(optimizer='Adam', loss='mse', metrics=['mae'])
    
    # make sure history object is returned by model.fit()
    history = model.fit(x, y, 
                        # enter your code here
                        #epochs=
                        #batch_size= 
                        verbose=0)
    
    return history, model

## 2. Use Talos for hyperparameter optimization <a name="two"></a> 

This part is quite simple. The Talos experiment just uses the <code> Scan()</code> command. In the following,
we will investigate different arguments for this routine. However, there are only five necessary arguments
that are required:
* train_data (often known as 'x')
* train_targets (often known as 'y')
* params (the dictionary 'param' that we have created before)
* model (the 'build_model' that we also have created before)
* experiment_name (name of a folder in which the output, a csv-file, is stored)

In [ ]:
scan_object = talos.Scan(x=train_data,
                         y=train_targets,
                         model=build_better_model,
                         experiment_name='find_optimal_params',
                         params=param,
                         round_limit=10)

If all different parameter configurations from the dictionary are used, the routine <code> Scan()</code> has to
evaluate 768 different models. Depending on the neural network architecture and the number of training samples,
this can be very time-consuming.

As a solution, Talos offers several routines to limit the number of model configurations that is evaluated. Two useful
commands of <code> Scan()</code> for this purpose are:

* <code> round_limit=10</code> which limits the number of model evaluations to the specified integer value (e.g. 
10 model evaluations in this example).
* <code> fraction_limit = 0.1</code> specifies the fraction of `params` that will be tested  (e.g. 10% of the 
configurations in this example).   

### Access the results through the <code>Scan</code> object 

Using <code> round_limit=10</code> we have investigated only ten parameter configurations (since the time
for this tutorial is limited). But how do I get the ten configurations that have been used and what is their
performance?

For this purpose, the results of the scan object can be directly accessed:

In [ ]:
# accessing the results data frame
scan_object.data


The output on my local computer is (the results should differ on your machine since you 
consider ten other random parameter configurations which we will explain later on)

<img src='https://raw.githubusercontent.com/DLR-SC/Hyperparameter_tutorial/master/img/talos_scan_output.png' 
width=1000px>

There are several hyperparameter configurations with a mean average error (mae) in the range 2-3 
(i.e. the average house pricing error is 2000-3000 dollar).
Since the number of training samples is only 400, the results of the mean average error 
have a high variance and should not be overestimated. Nevertheless, we already obtain a certain 
understanding of a sensible size of some of the hyperparameters (e.g. 10 epochs is certainly not
 enough since 'mae' becomes large then).
 

You already noticed that your results differ from mine. This is due to the usage of a 'random search'
strategy in Talos. More information on the search strategy are given 
in the summary details using <code>scan_object.details</code>.  

In [ ]:
# access the summary details
scan_object.details


From the 768 possible hyperparameter configurations Talos considers a random subset
of ten configurations (due to <code>round_limit=10</code>). This random subset 
is chosen with a 'Mersenne Twister' pseudorandom number generator. Other possible
random choices in Talos are e.g. 'Halton' or 'Sobol' quasi Monte Carlo sequences. 

Note: If there is no parameter such as  <code> round_limit=10</code> or
<code> fraction_limit = 0.1</code> the default optimization strategy 
is called 'grid search'. This means that all hyperparameter permutations in 
a given dictionary are processed. In most cases, this is not recommended
for anything but very small permutation spaces. 
Therefore, better use a 'random search' routine.

In addition to statistics and meta-data related with the Scan, the used data (x and y) 
together with the saved model and model weights for each hyperparameter permutation is stored in the Scan object. 

In [ ]:
# accessing the saved models which returns a list of models
scan_object.saved_models

In [ ]:
# accessing the saved weights for models which returns a list of weights
model_weights = scan_object.saved_weights

# weights of first model 
model_weights[0]

The Scan object can be further used, and is required, as input for Predict(), Evaluate(), and Deploy(). 
More about this in the corresponding sections below.

### Analysing the Scan results with <code>Reporting()</code> 

In the Scan process, the results are stored round-by-round in the corresponding experiment log which is a .csv file stored in the 
output directory (which is './find_optimal_params' our case). The Reporting() 
accepts as its source either a file name, or the Scan object. 

In [ ]:
# use Scan object as input
analyze_object = talos.Analyze(scan_object)

In [ ]:
# get the number of rounds in the Scan
analyze_object.rounds()

In [ ]:
# get the lowest result for any metric (if lower is better)
analyze_object.low('mae')

Next, we want to know the parameters of our best n=3 parameter runs. 
For this purpose we use <code> best_params</code>. It is alluring to consider the best 
result only (i.e. n=1 in the following cell) but keep in mind that due to the high
variance in the result, a sequence of more models should be considered.

The signature of <code> best_params()</code> is the following:
* The first argument (here 'mae') is the metric that is considered.
* The second argument is a list of metrics / loss functions that is not required here
* The third argument gives the 'n' best results
* Fourth argument: ascending | bool | Set to True when `metric` is to be minimized eg. loss or mae

In [ ]:
# get the best n=3 paramaters
analyze_object.best_params('mae', ['loss'], n=3, ascending=True)

It is very important to determine which of your hyperparameters has the largest impact on 
your performance metric. These are the parameters that should be further investigated. 

One solution to achieve this is to compute the linear correlation between your performance metric
'mae' and your hyperparamters. The correlation coefficient is +1 in the case of a perfect direct
(increasing) linear relationship, -1 in the case of a perfect decreasing (inverse) linear 
relationship and some values in (-1, 1) otherwise (indicating the degree of linear dependance
between the variables).

We employ <code> correlate()</code> for computing the correlation

In [ ]:
# get correlation for hyperparameters against a performance metric such as 'mae' (we exclude 'loss' since 
# this is not a hyperparameter)
analyze_object.correlate('mae', ['loss', 'round_epochs'])

Since you have only then different hyperparameter runs, the output is almost random. However, in practice 
with hundreds or thousand of different configurations the result is much less stochastic noise in this output.

<img src='https://raw.githubusercontent.com/DLR-SC/Hyperparameter_tutorial/master/img/talos_correlate.png' 
width=240px>

In my special case, 'epoch_number' and 'number_of_neurons' have the strongest inverse effect on 'mae'
(i.e. 'more epochs tend to lead to a lower mean average error'). The 'batch_size' seems to be less important. 

You can also plot a heatmap of this correlation using <code> plot_corr()</code>:

In [ ]:
# heatmap correlation
analyze_object.plot_corr('mae', ['loss', 'round_epochs'])

The correlation between variables in one viable way to find the most important
hyperparameters. A different approach is given by <code> plot_bars()</code>.
Using this function, the metric 'mae' is plotted for different values of the 
hyperparameters 'number_of_neurons', 'number_of_layers' and 'epoch_number'. As mentioned
before, 'mae' seems to decrease for larger values of these hyperparameters (in the
specified region of the dictionary).

The bar grid cotanins error bars if there is more than one result for the specified
hyperparameter set ('number_of_neurons', 'epoch_number', 'number_of_layers'). 

In [ ]:
# a four dimensional bar grid
analyze_object.plot_bars('number_of_neurons','mae', 'epoch_number', 'number_of_layers')

My current output plot is

<img src='https://raw.githubusercontent.com/DLR-SC/Hyperparameter_tutorial/master/img/talos_bar.png' 
width=1300px>

## Tasks ## 
__Excercise 3:__
 - Apply the described process to your Talos output. In more detail:
    * Find the hyperparameters that have the largest effect on the performance metric.
    * Create a 2nd hyperparameter dictionary that further investigates these important hyperparameters
    * Use <code> Scan() </code> to investigate the 2nd dictionary in more detail 
    (use larger values for round_limit than 10).
    * Try to find a close-to-optimal model for this specific regression problem. 

In [ ]:
# your new hyperparameter dictionary
param2 = {
    # enter your code here
}

In [ ]:
scan_object2 = talos.Scan(x=train_data,
                         y=train_targets,
                         model=build_better_model,
                         experiment_name='find_optimal_params',
                         # enter your code here
                         #params=
                         #round_limit=
                         )

In [ ]:
# if you want to see the full output of Scan()
scan_object2.data

Your best results are:

In [ ]:
# use Scan object as input
analyze_object2 = talos.Analyze(scan_object2)

# get the best n=3 paramaters
analyze_object2.best_params('mae', ['loss'], n=3, ascending=True)

In [ ]:
# lowest mae with the parameters from analyze_object2.best_params('mae', ['loss'], n=1, ascending=True)
analyze_object2.low('mae')

## Deploy close-to-optimal model

### Evaluating Models with <code>Evaluate()</code> 

Models can be evaluated with <code>Evaluate()</code> against a k-fold cross-validation 
(<code>fold=K</code> specifies the number of repetitions in <code>Evaluate()</code>). 
Ideally at least 50% of the data, or more if possible, is kept completely out of the <code>Scan</code> process and only exposed into Evaluate once one or more candidate models have been identified.

In [ ]:
evaluate_object = talos.Evaluate(scan_object2)

# returns a list with 'folds=10' outputs
all_mae_results =  evaluate_object.evaluate(test_data, test_targets, folds=10, metric='mae', task='continuous')

# this is the average 'mae' that gives an aedequate estimation of our model error (on the test data)
np.mean(all_mae_results)

Once a sufficiently performing model has been found, a deployment package can be easily created.

### Deploying Models with <code>Deploy()</code> 

Once the right model or models have been found, you can create a deployment package with <code>Deploy()</code> which is then easy to transfer to a production or other environment, send via email, or upload to shared remote location. Best model is automatically chosen based on a given metric ('val_acc' by default).

The Deploy package is a zip file that consist of: 

- details of the scan
- model weights
- model json
- results of the experiment
- sample of x data
- sample of y data

The <code>Deploy</code> package can be easily restored with <code>Restore()</code> which is covered in the next section.

In [ ]:
# creates a file waw_regression_deploy.zip (\approx 13 KB) in the local folder 
# the parameter 'asc' has to be true if lower means better (e.g. loss, mae) but false otherwise (e.g. accuracy)
talos.Deploy(scan_object=scan_object2, model_name='waw_regression_deploy', metric='mae', asc=True);

### Restoring Models with <code>Restore()</code>  

In [ ]:
waw_regression = talos.Restore('waw_regression_deploy.zip')

The <code>Restore</code> object now consists of the assets from the Scan object originally associated with the experiment, together with the model that had been picked as 'best'. The model can be immediately used for making prediction, or use in any other other way Keras model objects can be used.

In [ ]:
# What is the 'best' model that we use for predictions?
# this should be the same model as found with (better check this) 
# 'analyze_object2.best_params('mae', ['loss'], n=1, ascending=True)' from the previous cell 
waw_regression.model.get_config()

In [ ]:
# make predictions with the model
# if you are interested, compare the output with the ground truth 'test_targets'
waw_regression.model.predict(test_data)

In addition, for book keeping purpose, and for simplicity of sharing models with team members and other stakeholders, various attributes are included in the <code>Restore</code> object:

In [ ]:
# get the meta-data for the experiment
waw_regression.details

In [ ]:
# get the hyperparameter space boundary, these are the hyperparameter values that you have considered before
waw_regression.params

In [ ]:
# these are the results from the different runs of Scan()
waw_regression.results

That's almost all on Talos for today. Of course, there are several other routines that are provided by Talos. You find
the most recent version (0.6.4 in November 2019) on GitHub (https://github.com/autonomio/talos).  

### Outlook: a large numbers of hidden layers in Talos

You probably have noticed that our model architecture in 'def build_better_model()'
can only be used for a relatively low number of hidden layers (since you have to add each layer manually).
For a larger number of hidden layers, Talos provides a 'hidden_layers' model 
(https://github.com/autonomio/talos/blob/master/docs/Hidden_Layers.md). When 'hidden_layers' are used,
several parameters must be included in your parameter dictionary. These parameters are 
'dropout', 'shapes', 'hidden_layers', 'first_neuron' and 'activation'. 

An application for our problem might look like this:

In [ ]:
param = {'hidden_layers' : [1, 2],          # <--- required
         'first_neuron' : [8, 16, 32, 64],  # <--- required
         'dropout' : [0, 0.1, 0.2],         # <--- required
         'shapes': ['brick'],               # <--- required
         'activation': ['relu'],            # <--- required
         'epoch_number' : [10, 20, 40, 80],
         'optimizer' : ['Adam', 'rmsprop'],
         'batch_size' : [1, 2, 4, 8]}

In [ ]:
from keras import models
from keras import layers
from talos.utils import hidden_layers

def build_even_better_model(train_data, train_targets, val_data, val_targets, p):
    model = models.Sequential()
    
    hidden_layers(model, p, 1) # <--- the required arguments are used here
        
    model.add(layers.Dense(1))
    model.compile(optimizer='Adam', loss='mse', metrics=['mae'])
    
    # make sure history object is returned by model.fit()
    history = model.fit(train_data, train_targets, epochs=p['epoch_number'], batch_size=p['batch_size'], verbose=0)
    
    return history, model


## Bonus ## 
__Excercise 4:__
 - Use the feature 'hidden_layers' from Talos for a new hyperparameter search.



### 3. Guidelines on hyperparameter optimization <a name="three"></a> 

* Hyperparameter optimization (at least as described in this tutorial with Talos) is an iterative process. You define the
hyperparamter boundaries and start a search with <code>Scan()</code>. Using the results, you define new
hyperparameter boundaries and a second search, ..., ..., final search. 
* In most situations, it is not necessary to perform a full grid search on all possible hyperparameter
configurations (maybe in the order of thousands). Usually, it is sufficient to perform the scan on 
a random subset (10-30% )
* It is important to determine those hyperparameter that have the largest influence on your performance
metric. For this purpose, Talos provides routine such as <code>correlate()</code> assist you to find these hyperparameters.
* All different hyperparameter combinations are evaluated on a validation set. As usual, only your final (after several iterations)
model is evaluated with a test set that is independent of the validation set.
* Even though Talos can be used analyze thousands of hyperparameter configurations, it is important to
first get some kind of intuition/understanding which model architecture might be adequate (e.g. do you require 
fully connected neural networks?, convolutional neural networks?, what might be the rough number
of hidden layers?). Otherwise, since the hyperparameter space grows exponentially you will not be able 
to evaluate a relevant subset of your high-dimensional hypercube within a reasonable amount of time
(cf. Curse of Dimensionality).


Thank you for your participiation! <br>
We hope that you have enjoyed this tutorial.